In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def dominant_colors(image_path, n_colors=3):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB

    # Reshape image into a 2D array of pixels
    pixels = img_rgb.reshape(-1, 3)

    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(pixels)

    # Get the top dominant colors (in RGB and HSV)
    dominant_rgb = kmeans.cluster_centers_.astype(int)
    dominant_hsv = cv2.cvtColor(np.uint8([dominant_rgb]), cv2.COLOR_RGB2HSV)[0]

    return dominant_rgb, dominant_hsv


In [2]:
def color_temperature_proportion(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB

    # Tentukan batasan untuk warna hangat (R > G > B) dan dingin (B > G > R)
    warm_pixels = np.sum((img_rgb[:, :, 0] > img_rgb[:, :, 1]) & (img_rgb[:, :, 0] > img_rgb[:, :, 2]))
    cold_pixels = np.sum((img_rgb[:, :, 2] > img_rgb[:, :, 0]) & (img_rgb[:, :, 2] > img_rgb[:, :, 1]))

    total_pixels = img_rgb.shape[0] * img_rgb.shape[1]
    
    warm_percentage = (warm_pixels / total_pixels) * 100
    cold_percentage = (cold_pixels / total_pixels) * 100

    return warm_percentage, cold_percentage


In [3]:
def unique_colors_count(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB

    # Reshape the image and remove duplicates to count unique colors
    unique_colors = np.unique(img_rgb.reshape(-1, img_rgb.shape[2]), axis=0)

    return len(unique_colors)


In [4]:
def quantize_colors(image_path, n_clusters=16):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB

    # Reshape image into a 2D array of pixels
    pixels = img_rgb.reshape(-1, 3)

    # Apply KMeans to quantize the image colors
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    
    # Get the quantized colors (centroids)
    quantized_colors = np.unique(kmeans.labels_, return_counts=True)

    return len(quantized_colors[0])


In [ ]:
import os
import pandas as pd

def extract_all_features(image_path):
    # Mengambil fitur dari setiap fungsi
    dominant_rgb, dominant_hsv = dominant_colors(image_path)
    warm_percentage, cold_percentage = color_temperature_proportion(image_path)
    unique_colors = unique_colors_count(image_path)
    quantized_colors = quantize_colors(image_path)

    # Memastikan output berupa array 1D untuk RGB dan HSV
    return {
        'image_name': os.path.splitext(os.path.basename(image_path))[0],  # nama gambar tanpa ekstensi
        'dominant_rgb': dominant_rgb,  # Tetap berupa array 1D (3 nilai)
        'dominant_hsv': dominant_hsv,  # Tetap berupa array 1D (3 nilai)
        'warm_percentage': warm_percentage,
        'cold_percentage': cold_percentage,
        'unique_colors': unique_colors,
        'quantized_colors': quantized_colors
    }

def process_images_to_csv(folder_path, output_csv):
    image_features = []

    # Memproses semua gambar dalam folder
    for filename in sorted(os.listdir(folder_path), key=lambda x: (x.isdigit(), x)):
        # Hanya proses file gambar yang memiliki ekstensi .png, .jpg, atau .jpeg
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')) and filename != '.DS_Store':
            image_path = os.path.join(folder_path, filename)
            features = extract_all_features(image_path)
            image_features.append(features)

    # Membuat DataFrame dan menyimpan ke CSV
    df = pd.DataFrame(image_features)

    # Memisahkan fitur RGB dan HSV menjadi kolom individual
    # Pastikan bahwa setiap nilai RGB dan HSV adalah array 1D dengan 3 nilai
    rgb_values = []
    hsv_values = []

    for idx, row in df.iterrows():
        rgb_values.append(row['dominant_rgb'])
        hsv_values.append(row['dominant_hsv'])

    # Jika nilai dalam 'dominant_rgb' dan 'dominant_hsv' berupa list 1D yang valid
    rgb_df = pd.DataFrame(rgb_values, columns=[f'dominant_rgb_{i}' for i in range(3)])
    hsv_df = pd.DataFrame(hsv_values, columns=[f'dominant_hsv_{i}' for i in range(3)])

    # Gabungkan dengan DataFrame yang lain
    df = pd.concat([df.drop(['dominant_rgb', 'dominant_hsv'], axis=1), rgb_df, hsv_df], axis=1)

    # Simpan ke CSV
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV disimpan ke: {output_csv}")


In [17]:
folder_path = '../Datasets/datasets'  
output_csv = 'color_composition.csv'

process_images_to_csv(folder_path, output_csv)

ValueError: Must pass 2-d input. shape=(300, 3, 3)